# TFDS Dataset Installation

In [ ]:
!pip3 uninstall tensorflow_datasets
!pip3 install git+git://github.com/jingjing-shi/datasets

Uninstalling tensorflow-datasets-4.0.1:
  Would remove:
    /usr/local/bin/tfds
    /usr/local/lib/python3.6/dist-packages/tensorflow_datasets-4.0.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-datasets-4.0.1
  Cloning git://github.com/jingjing-shi/datasets to /tmp/pip-req-build-krvvx95h
  Running command git clone -q git://github.com/jingjing-shi/datasets /tmp/pip-req-build-krvvx95h
  Created wheel for tensorflow-datasets: filename=tensorflow_datasets-3.2.1_nightly-cp36-none-any.whl size=3495334 sha256=5368939526038ea3b86cf48bf72b5816f32d6aef964645a49bebac5a3aec5e9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-byl14ph3/wheels/b1/bf/25/112bd40633bdeb00a184d30650219d44d9523630a3c242c614
Successfully built tensorflow-datasets


In [35]:
import pydicom as dicom
import glob
import matplotlib.pyplot as plt
import os
import nibabel as nib
import math
import numpy as np
from tensorflow.data.experimental import sample_from_datasets
from tensorflow.image import central_crop
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import Model
from skimage.util import crop
from tensorflow.keras.optimizers import *
from tensorflow.keras import initializers
import tensorflow.keras.backend as K
from tensorflow.keras import metrics
import SimpleITK as sitk
from tensorflow.keras import models
import tensorflow as tf
import tensorflow_datasets as tfds
from datetime import datetime
from packaging import version
from utilities import *
from unet_model_file import *

In [ ]:
## I wrote a tfds dataset for BRATS2015, so we can load tfds data
data_dir = 'gs://duke-tfds'
# load both HGG and LGG. 80/20 train val split
# Init dataset

train_data, info = tfds.load('Brats2015',data_dir=data_dir,with_info=True, split='train[:80%]', 
                             read_config=tfds.ReadConfig(interleave_parallel_reads=tf.data.experimental.AUTOTUNE,interleave_block_length=8), shuffle_files=True)
val_data, info = tfds.load('Brats2015',data_dir=data_dir,with_info=True, split='train[20%:]', 
                           read_config=tfds.ReadConfig(interleave_parallel_reads=tf.data.experimental.AUTOTUNE, interleave_block_length=8), shuffle_files=True)

# Pre-process data 

In [ ]:
# Create training data [BATCH BEFORE MAPPING]
processed_train_data = train_data.map(lambda x: binarize(x),num_parallel_calls = tf.data.experimental.AUTOTUNE)
processed_train_data = processed_train_data.map(lambda x: cast(x),num_parallel_calls = tf.data.experimental.AUTOTUNE)
processed_train_data = processed_train_data.map(lambda x: standardize(x), num_parallel_calls = tf.data.experimental.AUTOTUNE)
processed_train_data = processed_train_data.map(lambda x: reshaping(x), num_parallel_calls = tf.data.experimental.AUTOTUNE)
#Create validation data
processed_val_data = val_data.map(lambda x: binarize(x), num_parallel_calls = tf.data.experimental.AUTOTUNE) 
processed_val_data = processed_val_data.map(lambda x: cast(x), num_parallel_calls = tf.data.experimental.AUTOTUNE)
processed_val_data = processed_val_data.map(lambda x: standardize(x), num_parallel_calls = tf.data.experimental.AUTOTUNE)
processed_val_data = processed_val_data.map(lambda x: reshaping(x), num_parallel_calls = tf.data.experimental.AUTOTUNE)

In [ ]:
# Examine sample data
ds_iter = iter(processed_train_data)
sample = next(ds_iter)
image = sample[0][0]
mask = sample[1][0]
fig, axes = plt.subplots(1,2, figsize=(10,10))
axes[0].imshow(tf.squeeze(image))
axes[0].set_title('Image')
axes[1].imshow(tf.squeeze(mask))
axes[1].set_title('Mask')

plt.show()

# Create model and train on training dataset 

In [37]:
unet = unet_model()

In [38]:
unet.summary()

Model: "Unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 240, 240, 1) 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 240, 240, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 240, 240, 32) 9248        conv2d_24[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 120, 120, 32) 0           conv2d_25[0][0]                  
_______________________________________________________________________________________________

In [ ]:
# Create a TensorBoard callback
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=0, mode='auto', 
    restore_best_weights=True
)

In [ ]:
# Fit on 2015 brats data
unet.fit(processed_train_data, epochs = 20, verbose =1, validation_data = processed_val_data,callbacks=[tensorboard_callback, stopping,checkpoint])